## SUPERCONDUCTIVITY EXAMPLE WITH ROOT

In a superconductivity experiment, a four-wire technique was used to simultaneously measure the voltage across a superconducting sample of unknown material. Using the experimental data from this experiment, you will calculate the resistance and determine the critical temperature $T_c$ for the superconducting sample, and determine its identity.

Superconductors are characterized by a critical temperature, Tc, below which they have zero resistance. In the broadest terms, there are two classes of superconductors. There are those with low values for TC, typically 10 K or less, and those with much higher TC. Some of the high-TC materials at atmospheric pressures can get up to 150 K while still being superconductors. 

The high-TC material you will examine in this experiment is one of two different cuprates. The first is YBa2Cu3O7. In this material the stoichiometric ratio of Y, Ba, and Cu is 1,2,3 and hence it is sometimes called a “123” superconductor. The number of Oxygen atoms present is key to the superconducting properties of the sample. It is also known as “YBCO”. The structure is related to that of perovskites.

The other sample is Bi-Sr-Ca-Cu-oxide. Several different compounds are present in the sample, but the one giving rise to the properties we will observe is BiSr2Ca2Cu3O10. It will show superconducting properties at even higher temperatures than the YBCO sample. The abbreviation BSCCO frequently gets pronounced as “Bis - co”.

Experimentally, Tc is most often taken to be at the transition point of the resistance vs. temperature plot - the onset of the transition (if you are warming up from LN2 temperature) back to normal resistance behavior. This temperature must be determined as accurately as possible. You must understand the limitations of the system and determine a reasonable uncertainty in your temperature results. When you compare your numbers to those available elsewhere (cite your sources), you must realize that our samples may be somewhat different and not look exactly like those published elsewhere. Lastly, even in the best samples and experiments there appears to be a natural width of a few degrees Kelvin which corresponds to the onset of superconductivity. This itself is not entirely understood and a bit at odds with the (easier to explain) low-temperature superconductors, which typically exhibit a “sharper” transition.

Your tasks are:

1) Plot Resistance vs Temperature with error bars.
2) Consider an analysis methods for identifying the important features of your plot. You may choose to take one or more numerical derivatives to locate the temperature of interest. You may also choose to fit an appropriate analytical function to your data, to allow extraction of appropriate fit parameters. I suggest fitting a sigmoid:

$$S(x) = \frac{L}{1+e^{-k(x-x_0)}}$$

Where L: Height, k: steepness, x0: center

3) Determine the onset temperature of the transition where the resistance begins to in- crease from zero, with uncertainty. Feel free to estimate your uncertainties as constant values measured by the Keithley 2000 multiplier or just use least count.
4) Compare your temperature to literature values for your superconducting sample. The onset temperature value should agree within reasonable uncertainties, but there may be differences. One plausible explanation for this relates to the size of the ceramic superconducting disk and that the temperature across it may not be uniform. Try to articulate why this might produce an effect in your measurements. Ultimately, you should be able to tell if our sample is YBCO or BSCCO.

The purpose of this exercise is to give you a first order of understanding how to work with experimental data. The source file you have needs to be cleaned first, as only the rows with three columns are the ones that contain valuable data (the other ones are noise). Moreover, I want you to have a first and simple use case of ROOT plotting tools, so you should not use matplotlib or any other python libraries, do all your plots with ROOT. Also, you cannot use any fitting method other than the ones that come with ROOT, but you can use Numpy and Pandas if you want for simple data management and operations, but not for fittings. 

### Solution - Code

In this first part of the code, the necessary libraries are imported and the data is organized by filtering only the relevant ones.

In [153]:
import ROOT
import pandas as pd
import numpy as np

# Read the file
archivo1 = '/mnt/c/Users/ROSARIO ALDAVA/Downloads/warmingdata.txt'
with open(archivo1, 'r') as file:
    lines = file.readlines()

# Process each line to split it into columns and remove whitespace
data1 = [line.strip().split() for line in lines]
# Filter rows with exactly three values
filtered_data = [row for row in data1 if len(row) == 3]
# Define column names
column_names = ['Current'   ,    'Voltage',       'Temperature']
# Create a DataFrame with the filtered data and assign column names
df = pd.DataFrame(filtered_data, columns=column_names)
# Save the DataFrame to a text file
output_file = '/mnt/c/Users/ROSARIO ALDAVA/Downloads/warningdata.txt'
df.to_csv(output_file, sep='\t', index=False)
# Display the first few records of the DataFrame
print(df.head())

          Current          Voltage      Temperature
0  +1.00058331E-0  +5.05230239E-07  +8.33532955E+01
1  +1.00052682E-0  +3.82320582E-07  +8.34320661E+01
2  +1.00058630E-0  +9.86293195E-07  +8.35132276E+01
3  +1.00059577E-0  +9.59596502E-07  +8.36069451E+01
4  +1.00056507E-0  +6.65685845E-07  +8.37417115E+01


In this section, the resistance is calculated using the known formula $$R= V/I$$.

In [154]:
# Leer los datos del archivo W2data.txt
df = pd.read_csv('/mnt/c/Users/ROSARIO ALDAVA/Downloads/W2.txt', sep='\t')
# Convert current and voltage columns to numerical values
df['Current'] = df['Current'].astype(float) # x10**{2} mA
df['Voltage'] = df['Voltage'].astype(float)

# Calculate resistance
df['Resistance'] = df['Voltage'] / df['Current']

# Save the DataFrame to a text file
output_file = '/mnt/c/Users/ROSARIO ALDAVA/Downloads/W2.txt'
df.to_csv(output_file, sep='\t', index=False)

print(f"The data has been successfully saved to the file {output_file}.")
# Display the DataFrame with the calculated resistance
print(df.head())


The data has been successfully saved to the file /mnt/c/Users/ROSARIO ALDAVA/Downloads/W2.txt.
   Current       Voltage  Temperature  Resistance
0  0.00001  5.052302e-07    83.353296    0.050494
1  0.00001  3.823206e-07    83.432066    0.038212
2  0.00001  9.862932e-07    83.513228    0.098572
3  0.00001  9.595965e-07    83.606945    0.095903
4  0.00001  6.656858e-07    83.741711    0.066531


A first attempt at fitting, setting a limit for a chosen sigmoid function.

In [175]:

# Read the data from the file W2data.txt.
df = pd.read_csv('/mnt/c/Users/ROSARIO ALDAVA/Downloads/W2.txt', sep='\t')

# Convert the resistance and temperature columns to numerical values.
resistance = df['Resistance'].values
temperature = df['Temperature'].values
voltage = df['Voltage'].values
#--------------------------------------------------------------------------------------------------
# Define sigmoid function
def sigmoid(x, par):
    # Ensure that the sigmoid function is within the limits
    if x[0] < min(temperature) or x[0] > max(temperature):
        return float('inf')  # Devolver infinito fuera del rango
    else:
        return par[0] / (1 + np.exp(-par[1] * (x[0] - par[2])))

# Create a TF1 with the sigmoid function.
fit_func = ROOT.TF1("fit_func", sigmoid, min(temperature), max(temperature), 3)

# Initial values for the fit parameters
initial_height = max(resistance) - min(resistance)
initial_slope = 1.0
initial_center = np.median(temperature)
fit_func.SetParameters(initial_height, initial_slope, initial_center)
#------------------------------------------------------------------------------------------------
# Create a TGraph
graph = ROOT.TGraph(len(temperature), temperature, resistance)

# Set axis titles
graph.GetXaxis().SetTitle('Temperature (K)')
graph.GetYaxis().SetTitle('Resistance')

# Set marker color and style
graph.SetMarkerColor(ROOT.kBlue)
graph.SetMarkerStyle(7)  # Circular marker
graph.SetLineColor(ROOT.kBlue)  # Set line color to match markers
graph.SetLineWidth(2)  # Set line width to 2 pixels

# Perform the fit.
graph.Fit("fit_func", "S")

# Fit parameters
fitted_height = fit_func.GetParameter(0)
fitted_slope = fit_func.GetParameter(1)
fitted_center = fit_func.GetParameter(2)

print(f"Fitted height: {fitted_height}")
print(f"Fitted slope: {fitted_slope}")
print(f"Fitted center: {fitted_center}")
#-------------------------------------------
# Draw the fitted sigmoid function.
fit_func.SetLineColor(ROOT.kRed)
fit_func.Draw("same")

# Canvas
c20 = ROOT.TCanvas('c20', 'Resistance vs Temperature', 1000, 600)

legend = ROOT.TLegend(0.1, 0.7, 0.3, 0.9)
legend.AddEntry(graph, "Experimental Data", "p")
legend.AddEntry(fit_func, "Fitted Sigmoid Function", "l")
legend.Draw()
#-------------------------
# Draw
graph.Draw('AP')  # 'AP' to draw points with error bars

c20.Draw()

Fitted height: 131.14711898361023
Fitted slope: 0.3227460954691912
Fitted center: 115.93492801804223
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      2979.33
NDf                       =          322
Edm                       =  2.43833e-06
NCalls                    =          112
p0                        =      131.147   +/-   0.28956     
p1                        =     0.322746   +/-   0.0050416   
p2                        =      115.935   +/-   0.0515578   


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c20


Now the errors are introduced, taking as reference the minimum readings from the datasheet of the Keithley 2000 multimeter, and propagating the error for the resistance.

In [172]:
# Read the data from the file W2data.txt
df = pd.read_csv('/mnt/c/Users/ROSARIO ALDAVA/Downloads/W2data.txt', sep='\t')

# Convert resistance and temperature columns to numerical values
resistance = df['Resistance'].values
temperature = df['Temperature'].values
voltage = df['Voltage'].values
current = df['Current'].values
# Calculating the uncertainty in resistance
# Assuming uncertainties in voltage and current are negligible
# You should replace these values with the actual uncertainties if available
delta_V = 0.1e-6  # Uncertainty in voltage (taken from datasheet)
delta_I = 100e-9  # Uncertainty in current (taken from datasheet)

# Propagation of uncertainty formula
delta_R = resistance * np.sqrt((delta_V / voltage) ** 2 + (delta_I / current) ** 2)

# Create a graph with TGraphErrors
graph = ROOT.TGraphErrors(len(resistance), temperature, resistance, np.full(len(temperature), 0.5), delta_R)
#print(f"Error Voltage: {delta_V}")
#print(f"Error Current: {delta_I}")
#print(f"Error Resistance: {delta_R}")
#---------------------------------------------------------------------------------------------------------
# Set axis titles
graph.GetXaxis().SetTitle('Temperature (K)')
graph.GetYaxis().SetTitle('Resistance (Ohm)')

# Set marker color and style
graph.SetMarkerColor(ROOT.kBlue)
graph.SetMarkerStyle(7)  # Circular marker
graph.SetLineColor(ROOT.kGreen)  # Set line color to match markers
graph.SetLineWidth(2)  # Set line width to 2 pixels
#--------------------------------------------------------------------------------------
# Definir la función sigmoide
def sigmoid(x, par):
    #Ensure that the sigmoid function is within bounds
    if x[0] < min(temperature) or x[0] > max(temperature):
        return float('inf')  # Return infinity outside the range
    else:
        return par[0] / (1 + np.exp(-par[1] * (x[0] - par[2])))
#-----------------------------------------------------

# Create a TF1 with the sigmoid function and set parameter limits
fit_func = ROOT.TF1("fit_func", sigmoid, min(temperature), max(temperature), 3)

# Calculate the mean of temperature and resistance data
mean_temperature = np.mean(temperature)
mean_resistance = np.mean(resistance)

# Set initial values for the fit parameters using the mean values
initial_height = max(resistance) - min(resistance)
initial_slope = 1.0
initial_center = mean_temperature
fit_func.SetParameters(initial_height, initial_slope, initial_center)

#----------------------------------------------------------------------------------------
# Perform the fit.
graph.Fit("fit_func", "S")

# Get the fit parameters
fitted_height = fit_func.GetParameter(0)
fitted_slope = fit_func.GetParameter(1)
fitted_center = fit_func.GetParameter(2)

# Print the fit parameters
#print(f"Fitted height: {fitted_height}")
#print(f"Fitted slope: {fitted_slope}")
#print(f"Fitted center: {fitted_center}")

# Draw the fitted sigmoid function
fit_func.SetLineColor(ROOT.kRed)
fit_func.Draw("same")

# Add legend
legend = ROOT.TLegend(0.1, 0.7, 0.3, 0.9)
legend.AddEntry(graph, "Experimental Data", "p")
legend.AddEntry(fit_func, "Fitted Sigmoid Function", "l")
legend.Draw()

# Create a canvas
c20 = ROOT.TCanvas('c20', 'Resistance vs Temperature', 1000, 600)

# Draw the graph
graph.Draw('AP')  # 'AP' to draw points with error bars

c20.Draw()

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      16628.8
NDf                       =          322
Edm                       =   4.2119e-06
NCalls                    =          494
p0                        =   0.00137636   +/-   2.17064e-07 
p1                        =     0.204293   +/-   0.000506859 
p2                        =      119.339   +/-   0.0374046   


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c20


In this second attempt, I add a cut to the data considering the interval of interest where I can find the critical temperature.

In [167]:
# Create a graph with the filtered data.
graph = ROOT.TGraphErrors(len(resistencia_filtrada), temperatura_filtrada, resistencia_filtrada, np.full(len(temperatura_filtrada), 0.5), delta_R[filtro])
graph.SetTitle("Resistance vs Temperature Graph")
# Set axis titles
graph.GetXaxis().SetTitle('Temperature (K)')
graph.GetYaxis().SetTitle('Resistance (Ohm)')

# Set marker color and style
graph.SetMarkerColor(ROOT.kBlue)
graph.SetMarkerStyle(7)  # Circular marker
graph.SetLineColor(ROOT.kGreen)  # Set line color to match markers
graph.SetLineWidth(2)  # Set line width to 2 pixels

# Definir la función sigmoide
def sigmoid(x, par):
    if x[0] < 100 or x[0] > 125:
        return float('inf')  # Return infinity outside the range
    else:
        return par[0] / (1 + ROOT.TMath.Exp(-par[1] * (x[0] - par[2])))

# Create a TF1 with the sigmoid function and set parameter limits.
fit_func = ROOT.TF1("fit_func", sigmoid, 90, 130, 3)

# Calculate the average of the temperature and resistance data.
mean_temperature = np.mean(temperatura_filtrada)
mean_resistance = np.mean(resistencia_filtrada)

# Initial values for the adjustment parameters using the mean values.
initial_height = max(resistencia_filtrada) - min(resistencia_filtrada)
initial_slope = 1.0
initial_center = mean_temperature
fit_func.SetParameters(initial_height, initial_slope, initial_center)

# Perform the fit.
graph.Fit("fit_func", "S")

# Get the fit parameters.
fitted_height = fit_func.GetParameter(0)
fitted_slope = fit_func.GetParameter(1)
fitted_center = fit_func.GetParameter(2)
# Print the fit parameters
print(f"Fitted height: {fitted_height}")
print(f"Fitted slope: {fitted_slope}")
print(f"Fitted center: {fitted_center}")
# Dibujar la función sigmoide ajustada
fit_func.SetLineColor(ROOT.kRed)
fit_func.Draw("same")

legend = ROOT.TLegend(0.1, 0.7, 0.4, 0.9);
legend.AddEntry(graph, "Experimental Data", "p");
legend.AddEntry(fit_func, "Fitted Sigmoid Function", "l");
legend.Draw();

# Canva
c20 = ROOT.TCanvas('c20', 'Resistence vs Temperature', 1000, 600)

# Draw
graph.Draw('AP')  # 'AP' para dibujar puntos con barras de error
c20.Draw()



Fitted height: 120.3722559564592
Fitted slope: 0.4330448064076381
Fitted center: 115.26447237987279
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      423.442
NDf                       =          104
Edm                       =  1.55767e-06
NCalls                    =          199
p0                        =      120.372   +/-   0.213833    
p1                        =     0.433045   +/-   0.00465589  
p2                        =      115.264   +/-   0.0586129   


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c20


In [177]:
# Perform the fit
fit_result = graph.Fit("fit_func", "S")

# Get the fit parameters
fitted_height = fit_func.GetParameter(0)
fitted_slope = fit_func.GetParameter(1)
fitted_center = fit_func.GetParameter(2)

# Get the fit parameter errors
error_height = fit_func.GetParError(0)
error_slope = fit_func.GetParError(1)
error_center = 0.5 #fit_func.GetParError(2) The least count of Keithley

# Compute the first derivative of the sigmoid function
def sigmoid_first_derivative(x, par):
    return par[0] * par[1] * np.exp(-par[1] * (x[0] - par[2])) / (1 + np.exp(-par[1] * (x[0] - par[2])))**2

# Create a TF1 with the first derivative of the sigmoid function
fit_func_first_derivative = ROOT.TF1("fit_func_first_derivative", sigmoid_first_derivative, min(temperature), max(temperature), 3)
fit_func_first_derivative.SetParameters(fitted_height, fitted_slope, fitted_center)

# Calculate the temperature where the first derivative changes sign
transition_temperature = None
transition_temperature_error = None
for temp in np.linspace(min(temperature), max(temperature), 1000):
    if sigmoid_first_derivative([temp], [fitted_height, fitted_slope, fitted_center]) > 0:
        transition_temperature = temp
        transition_temperature_error = error_center
        break

# Print the temperature of onset of transition and its uncertainty
print(f"Temperature of onset of transition: {transition_temperature:.3f} ± {transition_temperature_error:.3f} K")

# Use the fitted center as the estimated critical temperature
estimated_critical_temperature = fitted_center
estimated_critical_temperature_error = error_center
print(f"Estimated critical temperature: {estimated_critical_temperature:.3f} ± {estimated_critical_temperature_error:.3f} K")


Temperature of onset of transition: 83.353 ± 0.500 K
Estimated critical temperature: 115.935 ± 0.500 K
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      2979.33
NDf                       =          322
Edm                       =  1.94211e-08
NCalls                    =           36
p0                        =      131.147   +/-   0.28956     
p1                        =     0.322746   +/-   0.0050416   
p2                        =      115.935   +/-   0.0515578   


In [185]:
# Calcular el 1% de la resistencia total
umbral_resistencia = 0.01 * fitted_height

# Iterar sobre las temperaturas para encontrar el punto de transición
for temp in np.linspace(min(temperature), max(temperature), 1000):
    if temp > 100 and temp < 125:
        if abs(sigmoid_first_derivative([temp], [fitted_height, fitted_slope, fitted_center])) > umbral_resistencia:
            transition_temperature = temp
            transition_temperature_error = error_center
            break

# Imprimir la temperatura de inicio de la transición y su incertidumbre
if transition_temperature is not None:
    print(f"Temperatura de inicio de la transición: {transition_temperature:.3f} ± {transition_temperature_error:.3f} K")
else:
    print("Temperatura de transición no encontrada dentro del rango especificado.")


Temperatura de inicio de la transición: 105.415 ± 0.500 K


In [189]:
import numpy as np
import ROOT

# Perform the fit
fit_result = graph.Fit("fit_func", "S")

# Get the fit parameters
fitted_height = fit_func.GetParameter(0)
fitted_slope = fit_func.GetParameter(1)
fitted_center = fit_func.GetParameter(2)

# Get the fit parameter errors
error_height = fit_func.GetParError(0)
error_slope = fit_func.GetParError(1)
error_center = error_center = 0.5 #fit_func.GetParError(2) The least count of Keithley #fit_func.GetParError(2)

# Compute the first derivative of the sigmoid function
def sigmoid_first_derivative(x, par):
    return par[0] * par[1] * np.exp(-par[1] * (x[0] - par[2])) / (1 + np.exp(-par[1] * (x[0] - par[2])))**2

# Create a TF1 with the first derivative of the sigmoid function
fit_func_first_derivative = ROOT.TF1("fit_func_first_derivative", sigmoid_first_derivative, min(temperature), max(temperature), 3)
fit_func_first_derivative.SetParameters(fitted_height, fitted_slope, fitted_center)

# Calculate the temperature where the first derivative changes sign
transition_temperature = None
transition_temperature_error = None

# Calculate the 1% threshold for resistance
umbral_resistencia = 0.01 * fitted_height

# Iterate over temperatures to find the transition point
for temp in np.linspace(min(temperature), max(temperature), 1000):
    if temp > 100 and temp < 125:
        if abs(sigmoid_first_derivative([temp], [fitted_height, fitted_slope, fitted_center])) > umbral_resistencia:
            transition_temperature = temp
            transition_temperature_error = error_center
            break

# Print the temperature of onset of transition and its uncertainty
if transition_temperature is not None:
    print(f"Temperature of onset of transition: {transition_temperature:.3f} ± {transition_temperature_error:.3f} K")
else:
   print("Transition temperature not found within the specified range.")

# Use the fitted center as the estimated critical temperature
estimated_critical_temperature = fitted_center
estimated_critical_temperature_error = error_center
print(f"Estimated critical temperature: {estimated_critical_temperature:.3f} ± {estimated_critical_temperature_error:.3f} K")


Temperature of onset of transition: 105.415 ± 0.500 K
Estimated critical temperature: 115.935 ± 0.500 K
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      2979.33
NDf                       =          322
Edm                       =  5.13502e-11
NCalls                    =           38
p0                        =      131.147   +/-   0.28956     
p1                        =     0.322746   +/-   0.00504161  
p2                        =      115.935   +/-   0.0515578   


The sample approaches the value of the superconductor BSCCO's critical temperature $T_{c}$. However, that is the estimated theoretical value $115.935\pm 0.5$, but the one observed experimentally is the temperature at which the transition begins $105.415\pm 0.5$. That's why we take that value to estimate the percentage of error.

In [191]:

# Theoretical value of critical temperature
theoretical_value = 110  # In K
# Calculate the percentage error for onset of transition
percentage_error = abs((transition_temperature - theoretical_value) / theoretical_value) * 100

# Print the percentage error
print(f"Percentage error: {percentage_error:.2f}%")


Percentage error: 4.17%


Considering now the expected temperature obtained from the fit

In [192]:

# Theoretical value of critical temperature
theoretical_value = 110  # In K
# Calculate the percentage error
percentage_error = abs((estimated_critical_temperature - theoretical_value) / theoretical_value) * 100

# Print the percentage error
print(f"Percentage error: {percentage_error:.2f}%")


Percentage error: 5.40%


In [193]:
# Get the chi-squared value from the fit result
chi_squared = fit_result.Chi2()

# Get the number of degrees of freedom from the fit result
degrees_of_freedom = fit_result.Ndf()

# Calculate the reduced chi-squared value
reduced_chi_squared = chi_squared / degrees_of_freedom

# Print the reduced chi-squared value
print(f"Reduced chi-squared value: {reduced_chi_squared:.2f}")


Reduced chi-squared value: 9.25


The variation in temperature within the ceramic superconductor disk may be due to the lack of uniformity in thermal distribution throughout the material. This is because different sections of the disk may heat unevenly due to factors such as the disk's geometry, non-uniform thermal conduction, or irregular energy distribution in the sample.

In a homogeneous material, the temperature is expected to remain constant everywhere when heat is applied uniformly. However, in materials like ceramic superconductors, the chemical composition and crystal structure may vary across the material, affecting its ability to conduct and distribute heat uniformly. This can result in temperature gradients along the disk, where some areas may be cooler or hotter than others at any given time.

These thermal gradients can influence temperature measurements, especially when taken at a specific point in the material. Discrepancies in measured temperature may arise due to these local temperature variations, making it challenging to accurately determine the material's critical temperature. 
